In [3]:
import math
import sys
from typing import List
from functools import lru_cache, reduce, cmp_to_key
from itertools import starmap, accumulate, chain, islice, product, tee, pairwise
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
from importlib import reload
reload(arrays)
reload(trees)
reload(lists)
reload(graphs)
reload(search)
import arrays as ar
import trees as tr
import lists as ls
import graphs as g
from lists import Node
print()

In [4]:
def aggressive_cows(stalls: list, cows: int) -> int:
    "Return maximum possible distance between `cows` when placed into the `stalls`."
    stalls.sort()
    count = lambda d: lambda a, s: (a[0] + 1, s) if s - a[1] >= d else a
    gt = lambda d: cows > reduce(count(d), stalls, (1, stalls[0]))[0]
    return search.binary(gt, 1, stalls[-1] - stalls[0])

In [5]:
aggressive_cows([10, 1, 2, 7, 5], 3)

4